In [1]:
%load_ext autoreload
%autoreload 2

import dask
import satpy
from tempfile import gettempdir
from pathlib import Path
from dask.diagnostics import Profiler, ResourceProfiler, visualize
from datetime import datetime, timezone
from satpy import Scene, find_files_and_readers
import hdf5plugin  # noqa: F401

import warnings

dask.config.set(num_workers=20)
satpy.config.set({'cache_dir': str(Path(f"{gettempdir()}").joinpath('cache'))})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})
warnings.filterwarnings('ignore')

#debug_on()

In [3]:
dask.config.set(scheduler='threads') 


bbox = (-1600000, -2770000, 690000, -1040000)

comp = "ir_105"

p = Path("testfiles")
infiles = p.joinpath("in")
infiles.mkdir(exist_ok=True)
tod = p.joinpath("out")
tod.mkdir(exist_ok=True)

hr = True
if hr:
    infiles = infiles.joinpath("hr")
    tod = tod.joinpath("hr")
else:
    infiles = infiles.joinpath("nr")
    tod = tod.joinpath("nr")


files = find_files_and_readers(base_dir=infiles, reader="fci_l1c_nc")
with dask.config.set({"array.chunk-size": "32MiB"}):
    with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
        st = datetime.now(timezone.utc)
        #if not os.path.isfile(outf):
        if True:
            scn = Scene(files)
            scn.load([comp], generate=False)
            scn2 = scn.chunk(chunks=(696*2, 696*2))
           # scn.load(['ir_38'], calibration='radiance')
            #scn.load([comp], generate=False)
            scn3 = scn2.resample("euro1")
            scn3.save_dataset(comp, base_dir=tod,  fill_value=0)
            #scn3.save_dataset(comp, base_dir=tod, enhance=False, dtype=np.float32, fill_value=0)

        en = datetime.now(timezone.utc)
        print('\n', (en-st).total_seconds())
    visualize([prof, rprof], show=False, save=True, filename=str(tod.joinpath("frp_new.html")))


 20.334481
